In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Data

In [2]:
print("="*60)
print("ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ ДЛЯ LIGHTGBM")
print("="*60)

# 1. Загрузка данных
print("1. Загрузка данных...")
train_df = pd.read_csv('./data/train_processed.csv')
val_df = pd.read_csv('./data/val_processed.csv')

# 2. Разделение на признаки и таргет
print("2. Разделение данных...")
X_train = train_df.drop(columns=['TARGET'])
y_train = train_df['TARGET']
X_val = val_df.drop(columns=['TARGET'])
y_val = val_df['TARGET']

# 3. Исправление названий столбцов для LightGBM
print("3. Исправление названий столбцов...")

def fix_column_names(df):
    """Исправляет названия столбцов для совместимости с LightGBM"""
    df_fixed = df.copy()
    new_columns = {}
    
    for col in df_fixed.columns:
        # Заменяем все не-буквенно-цифровые символы на подчеркивание
        new_name = str(col)
        # Заменяем пробелы и основные проблемные символы
        for char in [' ', '(', ')', '[', ']', '{', '}', '/', '\\', ':', ',', '.', '-', '+', '*', '&', '^', '%', '$', '#', '@', '!', '?']:
            new_name = new_name.replace(char, '_')
        
        # Убираем множественные подчеркивания
        while '__' in new_name:
            new_name = new_name.replace('__', '_')
        
        # Убираем подчеркивания в начале/конце
        new_name = new_name.strip('_')
        
        # Если имя пустое или начинается с цифры
        if not new_name:
            new_name = f'feature_{col}'
        elif new_name[0].isdigit():
            new_name = f'f_{new_name}'
        
        new_columns[col] = new_name
    
    return df_fixed.rename(columns=new_columns)

X_train = fix_column_names(X_train)
X_val = fix_column_names(X_val)

print(f"   Переименовано {X_train.shape[1]} столбцов")
print(f"   Пример: '{list(train_df.columns)[0]}' -> '{X_train.columns[0]}'")

# 4. Оптимизация типов данных
print("4. Оптимизация типов данных...")

# Целевая переменная -> int8
y_train = y_train.astype(np.int8)
y_val = y_val.astype(np.int8)

# Бинарные признаки (0/1) -> int8
binary_mask = (X_train.nunique() <= 2) & (X_train.isin([0, 1]).all())
binary_cols = X_train.columns[binary_mask].tolist()

if binary_cols:
    X_train[binary_cols] = X_train[binary_cols].astype(np.int8)
    X_val[binary_cols] = X_val[binary_cols].astype(np.int8)
    print(f"   Бинарных признаков: {len(binary_cols)} -> int8")

# Остальные признаки -> float32
other_cols = [col for col in X_train.columns if col not in binary_cols]
if other_cols:
    X_train[other_cols] = X_train[other_cols].astype(np.float32)
    X_val[other_cols] = X_val[other_cols].astype(np.float32)
    print(f"   Непрерывных признаков: {len(other_cols)} -> float32")

# 5. Удаление признаков с нулевой дисперсией
print("5. Удаление признаков с нулевой дисперсией...")
zero_var_cols = X_train.columns[X_train.std() == 0]
if len(zero_var_cols) > 0:
    X_train = X_train.drop(columns=zero_var_cols)
    X_val = X_val.drop(columns=zero_var_cols)
    print(f"   Удалено {len(zero_var_cols)} признаков с нулевой дисперсией")
else:
    print("   Признаков с нулевой дисперсией не найдено")

# 6. Очистка памяти
print("6. Очистка памяти...")
del train_df, val_df
import gc
gc.collect()

# 7. Итоговые результаты
print("\n" + "="*60)
print("ИТОГОВЫЕ РАЗМЕРЫ ДАННЫХ")
print("="*60)

print(f"X_train: {X_train.shape} (память: {X_train.memory_usage().sum()/1024**2:.1f} MB)")
print(f"X_val:   {X_val.shape} (память: {X_val.memory_usage().sum()/1024**2:.1f} MB)")
print(f"y_train: {y_train.shape} (тип: {y_train.dtype})")
print(f"y_val:   {y_val.shape} (тип: {y_val.dtype})")

print(f"\nТипы признаков в X_train:")
for dtype in X_train.dtypes.unique():
    count = (X_train.dtypes == dtype).sum()
    print(f"  {dtype}: {count} признаков")

print(f"\nПроверка пропущенных значений:")
print(f"  X_train: {X_train.isna().sum().sum()}")
print(f"  X_val:   {X_val.isna().sum().sum()}")

print(f"\nПримеры названий признаков:")
print(f"  {list(X_train.columns[:3])}")

print("\n" + "="*60)
print("ДАННЫЕ ГОТОВЫ ДЛЯ LIGHTGBM!")
print("="*60)

ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ ДЛЯ LIGHTGBM
1. Загрузка данных...
2. Разделение данных...
3. Исправление названий столбцов...
   Переименовано 702 столбцов
   Пример: 'SK_ID_CURR' -> 'SK_ID_CURR'
4. Оптимизация типов данных...
   Бинарных признаков: 115 -> int8
   Непрерывных признаков: 587 -> float32
5. Удаление признаков с нулевой дисперсией...
   Удалено 5 признаков с нулевой дисперсией
6. Очистка памяти...

ИТОГОВЫЕ РАЗМЕРЫ ДАННЫХ
X_train: (184506, 697) (память: 432.5 MB)
X_val:   (61502, 697) (память: 144.2 MB)
y_train: (184506,) (тип: int8)
y_val:   (61502,) (тип: int8)

Типы признаков в X_train:
  float32: 587 признаков
  int8: 110 признаков

Проверка пропущенных значений:
  X_train: 0
  X_val:   0

Примеры названий признаков:
  ['SK_ID_CURR', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL']

ДАННЫЕ ГОТОВЫ ДЛЯ LIGHTGBM!


# Stacking

In [3]:
print("="*80)
print("СТЕКИНГ (STACKING) НА ОСНОВЕ ЛУЧШИХ МОДЕЛЕЙ")
print("="*80)

import joblib
import numpy as np
import pandas as pd
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix
import time

# 1. Загрузка лучших моделей из файлов
print("\n1. Загрузка сохраненных моделей...")

# Random Forest
rf_artifacts = joblib.load('./models/random_forest_best_model.pkl')
rf_model = rf_artifacts['model']
print(f"✅ Random Forest загружен (из {type(rf_artifacts).__name__})")

# LightGBM
lgb_artifacts = joblib.load('./models/lgb_final_model.pkl')
lgb_model = lgb_artifacts['model']
print(f"✅ LightGBM загружен")

# XGBoost
xgb_artifacts = joblib.load('./models/xgb_final_model.pkl')
xgb_model = xgb_artifacts['model']
print(f"✅ XGBoost загружен")

# CatBoost
cat_artifacts = joblib.load('./models/catboost_final_model_with_metrics.pkl')
cat_model = cat_artifacts['model']
print(f"✅ CatBoost загружен")

# 2. Создание списка базовых моделей для стекинга
print("\n2. Создание стекинг ансамбля...")

base_estimators = [
    ('random_forest', rf_model),
    ('lightgbm', lgb_model),
    ('xgboost', xgb_model),
    ('catboost', cat_model)
]

# Мета-модель (логистическая регрессия)
meta_model = LogisticRegression(
    random_state=42,
    max_iter=1000,
    class_weight='balanced',
    n_jobs=-1
)

# Создание стекинг модели
stacking_model = StackingClassifier(
    estimators=base_estimators,
    final_estimator=meta_model,
    cv=5,                # 5-кратная кросс-валидация для обучения мета-модели
    stack_method='predict_proba',  # Используем вероятности
    n_jobs=-1,            # Параллельные вычисления
    passthrough=False     # Не добавляем исходные признаки
)

print(f"   Базовые модели: {[name for name, _ in base_estimators]}")
print(f"   Мета-модель: LogisticRegression")
print(f"   Кросс-валидация: 5 фолдов")

# 3. Обучение стекинг модели
print("\n3. Обучение стекинг модели...")
print(f"   Размер данных: {X_train.shape}")
print("   Это может занять некоторое время...")

train_start = time.time()
stacking_model.fit(X_train, y_train)
train_time = time.time() - train_start

print(f"\n✅ Стекинг модель обучена за {train_time/60:.1f} минут")

СТЕКИНГ (STACKING) НА ОСНОВЕ ЛУЧШИХ МОДЕЛЕЙ

1. Загрузка сохраненных моделей...
✅ Random Forest загружен (из dict)
✅ LightGBM загружен
✅ XGBoost загружен
✅ CatBoost загружен

2. Создание стекинг ансамбля...
   Базовые модели: ['random_forest', 'lightgbm', 'xgboost', 'catboost']
   Мета-модель: LogisticRegression
   Кросс-валидация: 5 фолдов

3. Обучение стекинг модели...
   Размер данных: (184506, 697)
   Это может занять некоторое время...

✅ Стекинг модель обучена за 57.6 минут


In [5]:
print("="*80)
print("БЛОК 2: ОЦЕНКА МЕТРИК СТЕКИНГ МОДЕЛИ")
print("="*80)

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix
import numpy as np
import pandas as pd
import time

# 1. Предсказания на валидационной выборке
print("\n1. Предсказания на валидации...")
y_pred = stacking_model.predict(X_val)
y_pred_proba = stacking_model.predict_proba(X_val)[:, 1]

# 2. Расчет метрик
print("\n2. Расчет метрик...")
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='macro')
roc_auc = roc_auc_score(y_val, y_pred_proba)

# 3. Вывод результатов
print("\n" + "="*60)
print("РЕЗУЛЬТАТЫ СТЕКИНГ МОДЕЛИ")
print("="*60)

print(f"\nОсновные метрики:")
print(f"  Accuracy:   {accuracy:.4f}")
print(f"  F1 Macro:   {f1:.4f}")
print(f"  ROC-AUC:    {roc_auc:.4f}")

print("\nClassification Report:")
print(classification_report(y_val, y_pred, zero_division=0))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_val, y_pred)
print(cm)

print("\n" + "="*80)
print("ПЕРЕХОД К БЛОКУ 3: СОХРАНЕНИЕ МОДЕЛИ")
print("="*80)

БЛОК 2: ОЦЕНКА МЕТРИК СТЕКИНГ МОДЕЛИ

1. Предсказания на валидации...


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done 250 out of 250 | elapsed:    0.4s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done 250 out of 250 | elapsed:    0.3s finished



2. Расчет метрик...

РЕЗУЛЬТАТЫ СТЕКИНГ МОДЕЛИ

Основные метрики:
  Accuracy:   0.7081
  F1 Macro:   0.5478
  ROC-AUC:    0.7754

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.71      0.82     56537
           1       0.17      0.70      0.28      4965

    accuracy                           0.71     61502
   macro avg       0.57      0.70      0.55     61502
weighted avg       0.90      0.71      0.77     61502


Confusion Matrix:
[[40079 16458]
 [ 1497  3468]]

ПЕРЕХОД К БЛОКУ 3: СОХРАНЕНИЕ МОДЕЛИ


In [6]:
print("="*80)
print("БЛОК 3: СОХРАНЕНИЕ СТЕКИНГ МОДЕЛИ С МЕТРИКАМИ")
print("="*80)

import joblib
import os
import pandas as pd

# 1. Подготовка данных для сохранения
print("\n1. Подготовка артефактов для сохранения...")

stacking_artifacts = {
    # Сама модель
    'model': stacking_model,
    
    # Информация о базовых моделях
    'base_models': [name for name, _ in base_estimators],
    'meta_model': 'LogisticRegression',
    
    # Параметры обучения
    'training_time_minutes': train_time / 60 if 'train_time' in locals() else None,
    
    # Метрики на валидации
    'val_metrics': {
        'accuracy': round(accuracy, 4),
        'f1_macro': round(f1, 4),
        'roc_auc': round(roc_auc, 4),
        'confusion_matrix': cm.tolist(),
        'classification_report': classification_report(y_val, y_pred, zero_division=0, output_dict=True)
    },
    
    # Информация о данных
    'data_info': {
        'feature_count': X_train.shape[1],
        'training_samples': len(X_train),
        'validation_samples': len(X_val),
        'class_distribution_train': y_train.value_counts().to_dict(),
        'class_distribution_val': y_val.value_counts().to_dict()
    },
    
    # Временная метка
    'timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'model_type': 'StackingClassifier'
}

print("   Артефакты подготовлены:")
print(f"   - Базовая модель: да")
print(f"   - Метрики валидации: {len(stacking_artifacts['val_metrics'])} шт")
print(f"   - Информация о данных: {len(stacking_artifacts['data_info'])} шт")

# 2. Сохранение в файл
print("\n2. Сохранение модели в файл...")

output_path = './models/stacking_final_model.pkl'
joblib.dump(stacking_artifacts, output_path)

# Проверка размера файла
file_size = os.path.getsize(output_path) / 1024**2
print(f"   ✅ Модель сохранена: {output_path}")
print(f"   📦 Размер файла: {file_size:.1f} MB")

# 3. Итоговая информация
print("\n" + "="*80)
print("ИТОГОВАЯ ИНФОРМАЦИЯ О СТЕКИНГ МОДЕЛИ")
print("="*80)

print(f"\nМодель: Stacking Classifier")
print(f"Базовые модели: {', '.join([name for name, _ in base_estimators])}")
print(f"Мета-модель: LogisticRegression")
print(f"Время обучения: {train_time/60:.1f} минут" if 'train_time' in locals() else "Время обучения: N/A")

print(f"\nМетрики на валидационной выборке:")
print(f"  Accuracy:   {accuracy:.4f}")
print(f"  F1 Macro:   {f1:.4f}")
print(f"  ROC-AUC:    {roc_auc:.4f}")

print(f"\nConfusion Matrix:")
print(cm)

print("\n" + "="*80)
print("СТЕКИНГ МОДЕЛЬ УСПЕШНО СОХРАНЕНА!")
print("="*80)

БЛОК 3: СОХРАНЕНИЕ СТЕКИНГ МОДЕЛИ С МЕТРИКАМИ

1. Подготовка артефактов для сохранения...
   Артефакты подготовлены:
   - Базовая модель: да
   - Метрики валидации: 5 шт
   - Информация о данных: 5 шт

2. Сохранение модели в файл...
   ✅ Модель сохранена: ./models/stacking_final_model.pkl
   📦 Размер файла: 355.0 MB

ИТОГОВАЯ ИНФОРМАЦИЯ О СТЕКИНГ МОДЕЛИ

Модель: Stacking Classifier
Базовые модели: random_forest, lightgbm, xgboost, catboost
Мета-модель: LogisticRegression
Время обучения: 57.6 минут

Метрики на валидационной выборке:
  Accuracy:   0.7081
  F1 Macro:   0.5478
  ROC-AUC:    0.7754

Confusion Matrix:
[[40079 16458]
 [ 1497  3468]]

СТЕКИНГ МОДЕЛЬ УСПЕШНО СОХРАНЕНА!
